# DSBFによる多チャネル音源分離
実装中

In [ ]:
%%shell
git clone -b beta https://github.com/tky823/audio_source_separation.git
pip install soundfile

In [ ]:
%cd "/content/audio_source_separation/egs/bss-example/dsbf"

## データの準備
[CMU ARCTICデータベース](http://www.festvox.org/cmu_arctic/)の音声，および[Multi-Channel Impulse Response Database](https://www.iks.rwth-aachen.de/en/research/tools-downloads/databases/multi-channel-impulse-response-database/)のインパルス応答を用いて，多チャネルの混合音をシミュレーションする．

In [ ]:
%%shell
. ./prepare.sh

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from bss.beamform import DelaySumBeamformer

In [ ]:
plt.rcParams['figure.dpi'] = 200

窓長などについて
- $T_{60}=160$ [ms]の残響のインパルス応答を使用する．
- 空間がランク$1$である仮定から，フーリエ変換の窓長は，$4096$サンプル（$=256$ [ms]）としている．
- シフト長は，窓長の半分の$2048$サンプルとしている

In [ ]:
sr = 16000
sound_speed = 340
fft_size, hop_size = 4096, 2048

In [ ]:
n_bins = fft_size//2 + 1
frequency = np.arange(0, n_bins) * sr / fft_size

## 2音源分離

In [ ]:
degree_aew, degree_axb = 345, 75

In [ ]:
aew_mic0, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic0.wav".format(sr, degree_aew))
axb_mic0, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic0.wav".format(sr, degree_axb))
x_mic0 = aew_mic0 + axb_mic0

aew_mic1, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic1.wav".format(sr, degree_aew))
axb_mic1, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic1.wav".format(sr, degree_axb))
x_mic1 = aew_mic1 + axb_mic1

aew_mic2, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic2.wav".format(sr, degree_aew))
axb_mic2, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic2.wav".format(sr, degree_axb))
x_mic2 = aew_mic2 + axb_mic2

aew_mic3, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic3.wav".format(sr, degree_aew))
axb_mic3, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic3.wav".format(sr, degree_axb))
x_mic3 = aew_mic3 + axb_mic3

aew_mic4, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic4.wav".format(sr, degree_aew))
axb_mic4, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic4.wav".format(sr, degree_axb))
x_mic4 = aew_mic4 + axb_mic4

aew_mic5, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic5.wav".format(sr, degree_aew))
axb_mic5, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic5.wav".format(sr, degree_axb))
x_mic5 = aew_mic5 + axb_mic5

aew_mic6, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic6.wav".format(sr, degree_aew))
axb_mic6, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic6.wav".format(sr, degree_axb))
x_mic6 = aew_mic6 + axb_mic6

aew_mic7, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-{}_deg{}-mic7.wav".format(sr, degree_aew))
axb_mic7, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-{}_deg{}-mic7.wav".format(sr, degree_axb))
x_mic7 = aew_mic7 + axb_mic7

x = np.vstack([x_mic0, x_mic1, x_mic2, x_mic3, x_mic4, x_mic5, x_mic6, x_mic7])
n_sources, T = x.shape

### インパルス応答畳み込み後の音

In [ ]:
ipd.Audio(aew_mic0, rate=sr)

In [ ]:
ipd.Audio(axb_mic0, rate=sr)

### 混合音

In [ ]:
ipd.Audio(x[0], rate=sr)

In [ ]:
ipd.Audio(x[1], rate=sr)

### DSBFの実行

In [ ]:
_, _, X = ss.stft(x, nperseg=fft_size, noverlap=hop_size)

#### ステアリングベクトルの計算

In [ ]:
degrees = np.array([degree_aew, degree_axb]) / 180 * np.pi
x_source, y_source = np.sin(degrees), np.cos(degrees) # (n_sources,)
source_position = np.vstack([x_source, y_source]).transpose(1,0)
mic_position = np.array([[0.13, 0], [0.10, 0], [0.07, 0], [0.04, 0], [-0.04, 0], [-0.07, 0], [-0.10, 0], [-0.13, 0]])
A = np.exp(2j * np.pi * frequency[:,np.newaxis,np.newaxis] * np.sum(source_position * mic_position[:,np.newaxis,:], axis=2) / sound_speed) # (n_bins, n_channels, n_sources)

In [ ]:
beamformer = DelaySumBeamformer(steering_vector=A)
Y = beamformer(X)

In [ ]:
_, y = ss.istft(Y, nperseg=fft_size, noverlap=hop_size)
y = y[:,:T]

### 分離音

In [ ]:
ipd.Audio(y[0], rate=sr)

In [ ]:
ipd.Audio(y[1], rate=sr)